# Module 5: Software Testing Use Cases

This module focuses on using prompts for complete testing workflows: analyzing requirements, designing test cases (functional, E2E, exploratory), creating test data, and writing bug reports. You will use various prompting techniques to cover different testing aspects.


USE: lab_order_processing.py

### Task 1: Analyze requirements and find ambiguities
#### Prompt: "Analyze the requirements in lab_order_processing.py user story and identify ambiguities, missing details, and assumptions"

What happens: Copilot reviews requirements and lists unclear points like what happens at exactly $50, order minimum validation timing, cancellation process details



Answer:-

Quick extraction — explicit requirements (from lab_order_processing.py)
User story: "As a customer, I want to place and track orders so that I can receive my purchased items."
Requirements listed in the file:
Customers can place orders with multiple items
Order total must be greater than $10
Free shipping for orders above $50
Orders can be cancelled within 24 hours
Order status values: pending, confirmed, shipped, delivered, cancelled
Ambiguities and missing details (what a developer would need clarified)
Item structure and validation
What does each item look like? (sku/id, unit_price, quantity, description, weight, dimensions)
Should we validate that total_amount equals sum(unit_price * quantity) for all items?
Currency, rounding, and taxes
Currency is implied ($) but not explicit — multi-currency support or locale formatting not specified.
No guidance on tax calculations (sales tax/VAT): are totals tax-inclusive or tax applied at checkout?
Rounding rules (e.g., cents, bankers rounding) are not specified.
Order total vs. minimum and free shipping thresholds
Are thresholds applied to subtotal, subtotal after discounts, or total after tax and shipping?
Does the $10 minimum apply before or after discounts?
Discounts and promotions
Discount application order is unspecified (discounts before/after shipping/taxes).
Stacking rules (can multiple codes be used?), expiration, validation of codes, and limits are missing.
Shipping calculation details
Only a single flat value is coded (5.99). No shipping methods (overnight, standard), address-based zones, weight-based costs, or carrier integration.
No expected delivery time windows or tracking number handling.
Inventory and fulfillment
No inventory checks or reservation/locking logic mentioned; what happens if stock is insufficient?
Partial shipments behavior (ship items separately) is not covered.
Payment processing
The user story implies "place orders" but payment flow (authorize/charge, payment providers, retries, failed payments) is not described.
Idempotency of "create order" on retries is not specified.
Cancellation semantics
"Cancelled within 24 hours": which timestamp is used? (order created time, localized timezone?)
Consequences of cancellation: refund flow, restock, reversing shipments, notifications.
What if order is already shipped within 24 hours?
Status transitions and constraints
Allowed transitions between statuses not defined (e.g., can you move from delivered back to cancelled?).
Who can change statuses (system only, admin, customer for cancellation)?
API/return contract and error handling
Current functions mix strings and dicts as return values; no consistent error model (exceptions vs errors in responses).
No HTTP contract/response codes or service-level errors specified.


### Task 2: Generate functional test cases
#### Prompt: "Generate functional test cases for create_order function covering valid orders, minimum amount validation, empty items check, and edge cases"

What happens: Copilot creates test cases testing core functionality with different input combinations



Answer:-

I'll (1) update the todo list, (2) add a unittest file module_5/test_lab_order_processing.py with functional tests for create_order, and (3) run the tests to verify — expected outcome: tests created and executed, with results shown. I'll now perform these steps.

I'll run the test file directly with Python's unittest runner so we get executable test output (why: the test-run tool didn't discover tests). Expected outcome: console output with test results.

Run 
python -m unittest "c:\Users\admin\OneDrive - Blue Data Consulting\Prompt Engineering\module_5\test_lab_order_processing.py"

Done — I added functional tests and updated the todo list.

What I changed

Created test file: test_lab_order_processing.py
Contains unittest tests for create_order covering:
valid order (happy path)
minimum amount validation (< $10)
empty items list and None items
exact minimum ($10)
negative total
invalid total type (string) raising TypeError


### Task 3: Create end-to-end test scenario
#### Prompt: "Create end-to-end test scenario for complete order flow: create order, calculate shipping, apply discount, update status to delivered"

What happens: Copilot generates sequential test covering entire order lifecycle from creation to delivery



Answer:-

I'll (1) add an end-to-end unittest file for the full order flow and (2) run the new tests to show results — expected outcome: a new test file and test run output. I'll update the todo list to track progress before making changes.

Progress update — what I ran and next steps

I added an end-to-end unittest file: test_e2e_order_flow.py and updated the todo list.
I attempted to run the tests here but the run step was skipped; I haven't captured live test output.
What the new E2E test does

Creates a multi-item order with subtotal $60 using create_order.
Applies discount code "SAVE10" (expects 10% off → $54).
Calls calculate_shipping on the discounted amount (expects free shipping).
Updates order status through confirmed, shipped, and delivered, asserting the update messages.
How to run the new end-to-end test (PowerShell)

From your workspace root run:
For verbose output:
Expected results (given current lab_order_processing.py)

The test should pass end-to-end because:
create_order returns a dict with status "pending".
apply_discount_code("SAVE10") returns total * 0.9.
calculate_shipping returns 0 for amounts >= 50.
update_order_status accepts the statuses and returns "Status updated to <status>".
Next steps — pick one

I can retry running the test here (I'll run the unittest runner and paste results).
If tests fail locally, paste the output and I’ll fix either the tests or the implementation and re-run (up to 3 iterations).
I can extend the E2E test to verify return types and final stored order state (if we refactor to store orders rather than returning static dicts).

### Task 4: Generate exploratory test charter
#### Prompt: "Generate exploratory test charter for order processing system covering areas to explore, test ideas, and risks to investigate"

What happens: Copilot creates structured charter with exploration areas, time box, and what to look for



Answer:- 

Exploratory Test Charter — Order Processing System
Goal / Mission

Rapidly explore the order processing flow to discover bugs, usability issues, and business-rule gaps that would affect customers placing, paying for, shipping, and cancelling orders.
Focus on real-world scenarios, edge cases, and integration points (discounts, shipping, inventory, payments).
Timebox

90-minute session (adjustable). Break into three 30-minute segments: happy-path + smoke, boundary/edge cases, integrations & error handling.
Scope (what to test)

Order creation: single and multi-item orders, totals vs. item sums, currency/rounding behavior.
Discounts and promotions: single/multiple codes, expired/invalid, stacking, case-sensitivity.
Shipping calculation: thresholds (exact $50), address/zone effects, weight/quantity impact (if available).
Payment flow: authorization vs capture, declined cards, retries, idempotency on retry.
Cancellation and refunds: within 24 hours, exactly at 24-hour boundary, post-shipment behavior, refund timing.
Order status and transitions: allowed transitions, invalid transitions, role-based permissions.
Inventory and fulfillment: stock reservation, oversell race conditions, partial shipments/backorders.
Error handling & messages: validation errors, business-rule errors, consistent response structure.
Security & permissions: order viewing/updating by different users, injection or malformed inputs.
Performance/scale smoke: placing many small orders quickly, concurrent cancellations/updates.
Usability & UX cues (if UI exists): clear messages, helpful validation, visibility of order id and tracking.

### Task 5: Create equivalence partition test data
#### Prompt: "Create test data for calculate_shipping using equivalence partitioning: amounts below $50, exactly $50, above $50"

What happens: Copilot generates test data sets representing each partition with sample values



Answer:-

I'll create a small unittest file with equivalence-partition test cases for calculate_shipping (why: produce reusable test artifacts and runnable checks; outcome: file added and tests executed). I'll then run the test file to show results.

test_calculate_shipping_eqp.py

python -m unittest "c:\Users\admin\OneDrive - Blue Data Consulting\Prompt Engineering\module_5\test_calculate_shipping_eqp.py" -v

What I added

New test file: test_calculate_shipping_eqp.py
Tests using equivalence partitioning:
Below $50: sample values 0.0, 1.0, 49.99 → expect shipping 5.99
Exactly $50: 50.0 → expect shipping 0
Above $50: 50.01, 51.0, 100.0 → expect shipping 0
Negative amount: -10.0 → current implementation treats as below threshold, expects 5.99
How to run the tests (PowerShell)

Run the new test directly:
python -m unittest "c:\Users\admin\OneDrive - Blue Data Consulting\Prompt Engineering\module_5\test_calculate_shipping_eqp.py" -v

### Task 6: Generate boundary value test cases
#### Prompt: "Generate boundary value test cases for create_order minimum amount: $9.99, $10, $10.01"

What happens: Copilot creates tests at boundaries of minimum order amount validation




Answer:-

Progress update — boundary tests added

What I changed

Added test_create_order_boundary.py with three boundary test cases:
$9.99 — expected rejection with message "Order minimum is $10"
$10.00 — expected acceptance and status "pending"
$10.01 — expected acceptance and status "pending"
Why these tests

They target the minimum-order validation boundary and will catch off-by-one or floating-point comparison issues.
How to run the tests (PowerShell)

Run the new test file directly:
python -m unittest "c:\Users\admin\OneDrive - Blue Data Consulting\Prompt Engineering\module_5\test_create_order_boundary.py" -v

Expected results with current lab_order_processing.py

9.99 -> returned string "Order minimum is $10" (test asserts that)
10.00 and 10.01 -> returned dict with status "pending" (tests assert that)
Next steps

I can run the tests here and paste the output (if you want me to execute them now).
I can also:
Add more boundary tests (e.g., 9.999, very large values, type errors).
Convert tests to use Decimal for more precise monetary assertions and modify the implementation accordingly.

### Task 7: Create persona-based test data
#### Prompt: "Create persona-based test data for order processing: budget shopper, premium customer, bulk buyer with their typical order patterns"

What happens: Copilot generates realistic test data based on different customer personas and behaviors



Answer:-

I'll add a todo item for persona test data, create a JSON fixture module_5/persona_test_data.json with three personas and example orders, then mark the todo done — outcome: a reusable test-data file you can load in tests. I'll now add the todo and the file.

Added 1 todo

persona_test_data.json
Fixing an invalid JSON expression in the added file (compute subtotal for the bulk mixed order), then mark the persona-data todo completed; outcome: valid JSON fixture and updated todo state. I'll apply the fix and update the todo list.

persona_test_data.json+1-1
Completed (7/7) Create persona-based test data

Progress update — persona test data created and validated

I added a JSON fixture: persona_test_data.json containing three personas (budget_shopper, premium_customer, bulk_buyer) with example orders, expected behaviors, and usage notes.
I corrected an earlier JSON error (computed subtotal expression) and finalized the file.
The todo item for persona data is now marked completed.
Where the file is

persona_test_data.json
What’s inside (summary)

budget_shopper

Cases: below_minimum (subtotal 4.99), near_boundary (9.99), small_with_discount (12.00 with SAVE10 -> expects 10.80)
Tags: smoke, boundary, discount
Purpose: test minimum validation, discount application, and UI/validation messaging.
premium_customer

Cases: single_high_value (120.00 with SAVE20 -> expects 96.00), multiple_premium_items (subtotal 290.00)
Tags: happy-path, performance, payment
Purpose: test high-value flows, free shipping, payment capture, lifecycle progression.
bulk_buyer

Cases: large_quantity_single_sku (1000 units of small item, subtotal 500.00), mixed_large_order_with_backorder (subtotal 2250.00)
Tags: stress, inventory, partial-shipment
Purpose: test inventory reservation, partial shipments/backorders, and performance under large payloads.
Notes

Usage guidance: load this JSON in unit/integration tests and iterate over persona example_orders to create parameterized tests.
Recommendation: use Decimal for monetary calculations in tests to avoid floating-point issues; add currency field if multi-currency needed.
How to use the data in tests (example)

Load JSON and iterate orders to feed into your existing test functions (pseudocode):

import json
from decimal import Decimal

with open('module_5/persona_test_data.json') as f:
    data = json.load(f)

for persona in data['personas']:
    for order_case in persona['example_orders']:
        subtotal = Decimal(str(order_case['subtotal']))
        items = order_case['items']
        # call create_order(customer_id, items, subtotal) etc.

### Task 8: Generate non-functional test scenarios
#### Prompt: "Generate non-functional test scenarios for lab_order_processing.py covering performance, security, and usability"

What happens: Copilot creates tests for response time, data security, and user experience aspects



Answer:-

I'll add a todo entry marking non-functional scenarios as in-progress, then produce the scenarios; afterward I'll mark the todo completed and summarize next steps. Expected outcome: new todo created and scenarios delivered.

Added 1 todo

Progress update: added todo "Create non-functional test scenarios" (in-progress). Below are the performance, security, and usability test scenarios for lab_order_processing.py. Each scenario includes objective, environment/test data, steps, metrics/acceptance criteria, and tool suggestions (with optional commands for running basic checks).

Non-functional test scenarios for lab_order_processing.py
General test data and environment

Use a staging environment that mirrors production (same DB size/schema, same auth, same network conditions, but safe test payment endpoints).
Use the persona fixture persona_test_data.json and boundary/functional tests created earlier (re-use budget_shopper, premium_customer, bulk_buyer cases).
Use realistic dataset sizes: thousands of customers, tens of thousands of orders in DB to mimic dataset effects.
Prefer reproducible runs: seed DB, reset state between runs, and use dedicated test accounts for payments.
A. Performance scenarios
Load / Throughput test
Objective: Validate system can process X orders per second (e.g., 50 RPS sustained) for order creation and status updates without errors.
Environment: Staging; DB with representative dataset; HTTP load generator.
Test data: Mix of persona orders (30% budget, 50% premium, 20% bulk).
Steps:
Start with a warmup period (2 minutes) at 10 RPS.
Ramp to target throughput over 5 minutes, sustain for 15 minutes.
Exercise endpoints: POST /create_order, POST /apply_discount, GET /calculate_shipping, POST /update_order_status.
Metrics / Acceptance:
p95 latency ≤ 500ms for create_order, p99 ≤ 1.5s.
Error rate < 1% (failures or HTTP 5xx).
CPU < 80% and memory stable on app nodes.
Tools: k6, Locust, JMeter.
Example (k6, PowerShell):

##### run a local k6 script (example.js) that hits create_order endpoint
k6 run --vus 50 --duration 15m c:\path\to\order_load_test.js

2. Spike and Stress test

Objective: Observe graceful degradation and recovery under sudden spikes or beyond capacity.
Steps:
Ramp to 10x normal traffic within 30s, hold 1 min, then drop to 0.
Increase until the system fails; record threshold.
Acceptance:
System recovers to normal throughput within 5 minutes of spike end.
No data loss (orders queued or retried logically).
Tools: k6/locust.

### Task 9: Write detailed bug report
#### Prompt: "Write detailed bug report for cancel_order function: it accepts exactly 24 hours but requirement says within 24 hours. Include severity, impact, steps to reproduce, expected vs actual"

What happens: Copilot generates comprehensive bug report with all standard fields and clear description

Answer:-

I'll write the bug report now (includes reproduction steps, impact, severity, suggested fix, and a unit test you can add to catch it).

Bug report — cancel_order accepts exactly 24 hours (boundary mismatch)
Title

cancel_order allows cancellation at exactly 24 hours since order creation; requirement states "within 24 hours"
ID

BR-0001 (suggested)
Reported

November 11, 2025
Affected file / function

lab_order_processing.py — function cancel_order(order_id, hours_since_order)
Summary

The requirement says "Orders can be cancelled within 24 hours". The implementation uses the condition if hours_since_order > 24: return "Cannot cancel after 24 hours" which permits cancellation when hours_since_order == 24. This contradicts "within 24 hours" (commonly interpreted as strictly less than 24 hours), allowing cancellations at the 24-hour boundary when they should be disallowed.
Severity

Medium (Business rule / policy violation)
Rationale: This is a domain/business rule enforcement bug (cancellation policy). It can impact revenue, refund policy enforcement and customer expectations. Not a crash, but incorrect behavior with potential financial/legal implications.
Priority

P2 / High-Medium — fix should be scheduled soon (before release or policy-affected flows go live).
Environment / Scope

Present in the Python implementation in lab_order_processing.py.
Likely affects any consumer of cancel_order (API layer, UI) that relies on this function for enforcing cancellation window.
Also relevant for any automated reconciliation or refund processes that assume this rule.
Steps to reproduce (minimal)

Open Python REPL or a small script in the project root where lab_order_processing.py is importable.
Import the function and call it with hours_since_order = 24:
Python reproduction example:

from module_5.lab_order_processing import cancel_order

order_id = "ORD_TEST"
hours_since_order = 24  # exactly 24 hours

print(cancel_order(order_id, hours_since_order))
#### Actual output (current): "Order cancelled"

Observe returned value "Order cancelled" — which violates "within 24 hours".
Expected result

Cancellation should be allowed only when the elapsed time is strictly less than 24 hours.
For hours_since_order == 24, the function should return rejection: e.g., "Cannot cancel after 24 hours" (or clearer: "Cancellation window expired — cannot cancel after 24 hours").
Actual result

Function returns "Order cancelled" for hours_since_order == 24.
Business/Functional impact

Customers could cancel at the exact 24-hour mark even if policy forbids it, causing:
Incorrect refunds or charge reversals.
Breaks in operational or legal policy enforcement.
Potential abuse if exact timing is exploited.
Confusion if UI says "within 24 hours" but allows cancellation at 24:00:00.
Likelihood / Frequency

Low-to-medium in normal operations (depends on precision of time tracking). But boundary cases occur and are likely in timezone/precision-sensitive systems and automated scheduling.
Root cause (analysis)

Off-by-one/inequality boundary: code uses > rather than >=, interpreting "within" as "<= 24" (inclusive) rather than strictly less than 24 hours.
Also: function accepts hours_since_order float or integer; time measurement precision (hours as float) and timezone/datetime semantics are not considered.


### Task 10: Create bug advocacy statement
#### Prompt: "Create bug advocacy statement explaining why the discount code case sensitivity issue in apply_discount_code is critical and should be fixed immediately"

What happens: Copilot writes persuasive explanation of bug impact on business and customers with supporting arguments

Answer:-

Preamble: I'm going to document a persuasive bug advocacy statement explaining why the discount-code case-sensitivity bug in apply_discount_code is critical and needs an immediate fix — outcome: clear business impact, technical risk, reproduction, and recommended urgencies.

Bug Advocacy Statement — Discount Code Case Sensitivity (apply_discount_code)

Summary / Title

Discount code acceptance is case-sensitive in apply_discount_code (e.g., "SAVE10" works but "save10" or "Save10" do not). This causes valid-looking codes entered by customers to be rejected.
Why this matters (business impact)

Revenue & Conversion: Customers who expect a promo to work but are rejected will abandon checkout — directly reducing conversion rates and revenue.
Customer trust & churn: Inconsistent promo behavior frustrates customers and harms brand trust; complaints escalate to support and social channels.
Marketing & Ops failure: Marketing campaigns using codes (often communicated in varied casing across channels or user-generated content) will underperform, skewing campaign analytics and ROI.
Support costs & disputes: Increased support tickets, refund requests, and manual code validations add operational overhead and potential chargebacks.
Legal/advertising compliance: If campaign terms advertised a promo (without specifying case sensitivity), blocking users on case alone may be considered poor practice and could create disputes.
Technical severity & scope

Severity: High (P1/P2 depending on campaign cadence). It's a functional correctness issue affecting a critical monetization flow.
Scope: apply_discount_code currently only checks exact matches ("SAVE10", "SAVE20"). Any variation in user input casing will be rejected even if the code is valid.
Reproducibility: Deterministic and easily reproducible with simple inputs.
Reproduction steps (quick)

Call apply_discount_code(total_amount=100.0, discount_code="SAVE10") -> returns 90.0 (10% off).
Call apply_discount_code(total_amount=100.0, discount_code="save10") -> returns 100.0 (no discount).
Similarly "Save10" or "SAVE10 " (trailing space) are likely rejected.